# This example demonstrate how to train region classification (standart image classification model).

In [1]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
# os.environ["CUDA_VISIBLE_DEVICES"] = ""  # For CPU inference
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [2]:
import os
import sys
import warnings
from matplotlib import pyplot as plt
warnings.filterwarnings('ignore')

# Nomeroff-Net path
NOMEROFF_NET_DIR = os.path.abspath('../')
sys.path.append(NOMEROFF_NET_DIR)

In [3]:
plt.rcParams["figure.figsize"] = (10, 10)

In [4]:
%matplotlib inline 

In [5]:
# auto download latest dataset
from NomeroffNet.tools import modelhub

# auto download latest dataset
info = modelhub.download_dataset_for_model("numberplate_orientations")
PATH_TO_DATASET = info["dataset_path"]
PATH_TO_TMP_CUSTOM_DATASET = f'{PATH_TO_DATASET}_custom'

# local path dataset
#PATH_TO_DATASET = os.path.join(NOMEROFF_NET_DIR, "./data/dataset/OptionsDetector/numberplate_options_example")

In [6]:
PATH_TO_DATASET

'/var/www/nomeroff-net24/NomeroffNet/tools/../../data/./dataset/InverseDetector/numberplate_orientations/autoriaNumberplateOrientationDataset-2021-09-12'

In [7]:
PATH_TO_TMP_CUSTOM_DATASET

'/var/www/nomeroff-net24/NomeroffNet/tools/../../data/./dataset/InverseDetector/numberplate_orientations/autoriaNumberplateOrientationDataset-2021-09-12_custom'

In [8]:
DATASET_NAME = "inverse"
VERSION = "2021_09_12_pytorch_lightning"

RESULT_PATH = os.path.join(NOMEROFF_NET_DIR, "./data/models/", 'numberplate_{}_{}.ckpt'.format(DATASET_NAME, VERSION))

In [9]:
from NomeroffNet import InverseDetector

orientations = [
    "0", 
    "180"
]


In [10]:
# definde your parameters
class MyInverseClassificator(InverseDetector):
    def __init__(self):
        InverseDetector.__init__(self)
        
        # outputs
        self.orientations = orientations
        
        self.epochs           = 100
        self.batch_size       = 64
        
        # count gpu for train
        self.gpus = 1
        
        self.height         = 64
        self.width         = 295


In [12]:
# initialize region detector.
inverseClassificator = MyInverseClassificator()
inverseClassificator.prepare(PATH_TO_DATASET, verbose=1, num_workers=10)

START PREPARING
DATA PREPARED


In [13]:
# train
lr_finder = inverseClassificator.tune()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


TypeError: 'int' object is not subscriptable

In [ ]:
# Results can be found in
lr_finder["lr_find"].results

# Plot with
fig = lr_finder["lr_find"].plot(suggest=True)
fig.show()

In [ ]:
# train
model = npClassificator.train()

In [ ]:
npClassificator.prepare(PATH_TO_DATASET, verbose=1)

In [ ]:
npClassificator.test()

In [ ]:
npClassificator.save(RESULT_PATH)

# Classification Report

In [ ]:
npClassificator.load(RESULT_PATH)

In [ ]:
from NomeroffNet.data_modules.data_loaders import ImgGenerator
from sklearn.metrics import classification_report
import torch

def options_classification_report(npClassificator, 
                                  path_dir=os.path.join(PATH_TO_TMP_CUSTOM_DATASET, "test"), 
                                  mode_torch="gpu"):
    imageGenerator = ImgGenerator(
        path_dir,
        npClassificator.width,
        npClassificator.height,
        npClassificator.batch_size,
        [len(npClassificator.class_region), len(npClassificator.count_lines)])
    imageGenerator.build_data()
    gen = imageGenerator.path_generator()
    
    out_idx_reg_all = []
    out_idx_line_all = []
    label_idx_reg_all = []
    label_idx_line_all = []
    for i, (img_paths, inputs, labels) in enumerate(gen, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = torch.from_numpy(inputs)
        if mode_torch == "gpu":
            inputs = inputs.cuda()
            npClassificator.model.cuda()
        with torch.no_grad():
            outputs = npClassificator.model(inputs)
        label_reg = torch.from_numpy(labels[0])
        label_cnt = torch.from_numpy(labels[1])
        if mode_torch == "gpu":
            label_reg = label_reg.cuda()
            label_cnt = label_cnt.cuda()
        
        out_idx_reg = torch.max(outputs[0], 1)[1].cpu().numpy()
        out_idx_line = torch.max(outputs[1], 1)[1].cpu().numpy()
        
        label_idx_reg = torch.max(label_reg, 1)[1].cpu().numpy()
        label_idx_line = torch.max(label_cnt, 1)[1].cpu().numpy()
        
        out_idx_reg_all.extend(out_idx_reg)
        out_idx_line_all.extend(out_idx_line)
        label_idx_reg_all.extend(label_idx_reg)
        label_idx_line_all.extend(label_idx_line)
    #print("out_idx_reg_all", out_idx_reg_all)
    print("\n\nclass region classification report:")
    print(classification_report(label_idx_reg_all, 
                                out_idx_reg_all, 
                                target_names=npClassificator.class_region))
    print("\n\ncount lines classification report:")
    print(classification_report(label_idx_line_all, 
                                out_idx_line_all, 
                                target_names=npClassificator.count_lines))

In [ ]:
options_classification_report(npClassificator)

# Check small accuracy images

In [ ]:
npClassificator.load(RESULT_PATH)

In [ ]:
%matplotlib inline

In [ ]:
from NomeroffNet.data_modules.option_img_generator import ImgGenerator
import torch
import json
import cv2
import matplotlib.pyplot as plt

def show_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()

def small_acc(npClassificator, 
              path_dir=os.path.join(PATH_TO_DATASET, "test"), 
              mode_torch="gpu", 
              min_acc=0.75):
    imageGenerator = ImgGenerator(
        path_dir,
        npClassificator.width,
        npClassificator.height,
        npClassificator.batch_size,
        [len(npClassificator.class_region), len(npClassificator.count_lines)])
    print("start imageGenerator build")
    imageGenerator.build_data()
    print("end imageGenerator build")
    gen = imageGenerator.path_generator()
    for i, (img_paths, inputs, labels) in enumerate(gen, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs = torch.from_numpy(inputs)
        if mode_torch == "gpu":
            inputs = inputs.cuda()
        outputs = npClassificator.model(inputs)
        label_reg = torch.from_numpy(labels[0])
        label_cnt = torch.from_numpy(labels[1])
        if mode_torch == "gpu":
            label_reg = label_reg.cuda()
            label_cnt = label_cnt.cuda()
        out_idx_reg = torch.max(outputs[0], 1)[1].cpu().numpy()
        out_idx_line = torch.max(outputs[1], 1)[1].cpu().numpy()
        
        label_idx_reg = torch.max(label_reg, 1)[1].cpu().numpy()
        label_idx_line = torch.max(label_cnt, 1)[1].cpu().numpy()
        
        # check reg
        for o, l, p, tens in zip(out_idx_reg, label_idx_reg, img_paths, outputs[0]):
            if o != l:
                print("[WRONG REGION PREDICTED] pred", o, l, p)
                show_img(p)
            elif tens[l] < min_acc:
                print("[SMALL ACC REGION PREDICTED]  pred", o, l, tens[l], p)
                show_img(p)
        # check line
#         for o, l, p, tens in zip(out_idx_line, label_idx_line, img_paths, outputs[1]):
#             if o != l:
#                 print("[WRONG LINE PREDICTED]  pred", o, l, p)
#                 show_img(p)
#             elif tens[l] < min_acc:
#                 print("[SMALL ACC LINE PREDICTED]  pred", o, l, tens[l], p)
#                 show_img(p)

In [ ]:
small_acc(npClassificator, os.path.join(PATH_TO_DATASET, "test"))

In [ ]:
small_acc(npClassificator, os.path.join(PATH_TO_DATASET, "train"))

In [ ]:
small_acc(npClassificator, os.path.join(PATH_TO_DATASET, "val"))